In [2]:
!pip install --user python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)


pd.set_option('display.max_columns', None)

In [5]:
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")
KEY = os.environ.get("KEY")
HEADER = {"Authorization": f"Bearer {KEY}"}
BASIC_URL = "https://open.faceit.com/data/v4/"

In [7]:

#used to retreive time registered and elo
def retreive_player_by_id(id: str) -> dict:
    response = shorter_url(url=f"players/{id}")
    return response
# used to shorted the URL and not to specify header every time
def shorter_url(url = "") -> dict:
    request = BASIC_URL + url
    return requests.get(request, headers = HEADER).json()

#gets number of games for paginaition
def get_number_of_games_played(id: str) -> int:
    response = int(shorter_url(url = f"players/{id}/stats/csgo")["lifetime"]["Matches"])
    return response

#retreived time registered
def get_time_registered(id: str) -> str:
    response = retreive_player_by_id(id)
    return response["activated_at"]


#retreived elo and level
def get_level_elo(id: str) -> int:
    response = retreive_player_by_id(id)
    return (response["games"]["csgo"]["skill_level"], response["games"]["csgo"]["faceit_elo"])


#retreived all stats from all matches
def retreive_player_game_stats(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
    return response_array


# use for myself once only
def retreive_player_history(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    registered = get_time_registered(id)
    registered = int(pd.Timestamp(registered).timestamp())
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response[1::])
    return response_array




In [2]:
MatchHistory = retreive_player_history(FACEIT_PLAYERID)
with open(f"../data/raw/Match History/{FACEIT_PLAYERID}_match_history.json", "w") as file:
    json.dump(MatchHistory, file )   

In [11]:
GamesStats = retreive_player_game_stats(FACEIT_PLAYERID)
with open(f"../data/raw/Game Stats/{FACEIT_PLAYERID}_games_stats.json", "w") as file:
    json.dump(GamesStats, file )

In [51]:
def get_all_players_id():
    with open(f"../data/raw/Match History/{FACEIT_PLAYERID}_match_history.json", "r") as file:
        # df = pd.json_normalize(file, max_level=4)
        obj = json.load(file)
    list_2D = (list(map(lambda x: x["playing_players"], obj)))
    list_1D = {item for sub_list in list_2D for item in sub_list}

    return list_1D

In [52]:
players_IDs = get_all_players_id()

In [101]:
def download_player_data(id: str) -> None:
    data = retreive_player_game_stats(id)
    with open(f"../data/raw/Game Stats/{id}_games_stats.json", "w") as file:
        json.dump(data, file)

In [102]:
# for id in sample:
#     download_player_data(id)

In [88]:
# with open("../data/raw/Game Stats/Game Stats/40476548-200a-4e2b-8db2-0d8c98765efc_games_stats.json", "r") as file:
#     df = json.load(file)

In [89]:
# df = pd.json_normalize(df)

In [4]:
def rename_cols(text: str):
    text = text.split(".")[1]
    text = text.split(" ")
    text = "".join(list(map(str.capitalize, text)))
    return text



In [15]:
def create_df_from_json():
    df = pd.DataFrame()
    for ID in list(players_IDs):
        path = f"../data/raw/Game Stats/{ID}_games_stats.json"
        with open(path, "r") as file:
            obj = json.load(file)
            obj = pd.json_normalize(obj)
        obj = obj.rename(columns=rename_cols)
        df = pd.concat([df, obj], ignore_index=True)
    return df

def make_pickle(df: pd.DataFrame):
    df.to_pickle("dataCompressed.pkl", compression="zip")
